In [3]:
import warnings; warnings.simplefilter('ignore')
import ssl
from requests.exceptions import ConnectionError
from http.client import RemoteDisconnected
from urllib3.exceptions import ProtocolError
from ssl import SSLError
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.layers import Dense, LSTM,Dropout,Input,Flatten,Activation,LeakyReLU
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard 
import tensorflow.keras
import matplotlib.pyplot as plt
import nsepy
import pprint
import os
from datetime import date,timedelta
import datetime
import mplcursors
plt.style.use('fivethirtyeight')
from tqdm import tqdm
import pickle
from sklearn import preprocessing
import livelossplot
import wandb
# from wandb.keras import WandbCallback
from keras_tqdm import TQDMNotebookCallback
import tensorflow as tf
from IPython.display import clear_output
import seaborn as sns
homediraddr='/home/hemangjoshi37a_gmail_com'

In [25]:
class One_trian_stk_class:
    def __init__(self, stkname, insttkn,nbatch,nepochs):
        self.name = stkname
        self.insttkn = insttkn
        self.nepochs=nepochs
        self.nbatch=nbatch
        
    def load(self):
        filecheckpath =homediraddr+'/pfiles/'+self.name+'_'+'DF'+'.p'
        if(os.path.isfile(filecheckpath)):
            with open(filecheckpath, 'rb') as fp:
                self.df = pickle.load(fp)
            print('Loaded : '+self.name+'_'+'DF'+'.p')

        else:
            old_lst=[]
            interval='5minute'
            todaydt=datetime.date.today()
            hud_ago=todaydt-timedelta(days=99)
            to_date=datetime.date.isoformat(todaydt)
            from_date=datetime.date.isoformat(hud_ago)
            print('Getting Stock data : '+self.name)
            for i2 in tqdm(range(22)):
                new_lst = kite.historical_data(self.insttkn, from_date, to_date, interval,continuous=False)
                old_lst = new_lst + old_lst
                todaydt=todaydt-timedelta(days=100)
                hud_ago=hud_ago-timedelta(days=100)
                to_date=datetime.date.isoformat(todaydt)
                from_date=datetime.date.isoformat(hud_ago)
            locals()[self.name+'_DF']=pd.DataFrame(old_lst)
            with open(homediraddr+'/pfiles/'+self.name+'_'+'DF'+'.p', 'wb') as fp:
                pickle.dump(self.name+'_DF', fp, protocol=pickle.HIGHEST_PROTOCOL)
            print('Saved file '+homediraddr+'/pfiles/'+self.name+'_'+'DF'+'.p')
            print('Loaded : '+self.name+'_'+'DF'+'.p')

        with open('scale_dict.p', 'rb') as fp:
            scale_dict = pickle.load(fp)
            print('Loaded scale_dict')

        if(self.name not in scale_dict):
            data = locals()[self.name+'_DF'].filter(['close'])  #old_line
            dataset = data.values
            training_data_len = math.ceil( len(dataset) * .97 )
            scaler = MinMaxScaler(feature_range=(0,1))
            scaler=scaler.fit(dataset)
            scale_dict[self.name]=scaler
            with open('scale_dict.p', 'wb') as fp:
                pickle.dump(scale_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)
            print(self.name+'_DF'+' added to scale_dict.')
            self.scaler=scale_dict[self.name]
            print('Loaded scale_dict')
        else:
            self.scaler=scale_dict[self.name]
            print('Loaded scale_dict')
            
        filecheckpath =homediraddr+'/stkwise_models/'+self.name+'.h5'
        if(os.path.isfile(filecheckpath)):
            self.model=load_model(filecheckpath)
            print('model loaded : '+filecheckpath)
        else:
            self.model=load_model(homediraddr+'/figs/models/44.h5')
            print('model loaded : '+homediraddr+'/figs/models/44.h5')
            
    def xtrain_def():
        dataset = self.df.filter(['close']).values
        training_data_len = math.ceil( len(dataset) * .97 )
        scaled_data = self.scaler.transform(dataset)
        x_train = []
        y_train = []
        train_data = scaled_data[0:training_data_len , :]
        for i in range(60, len(train_data)):
          x_train.append(train_data[i-60:i, :])
          y_train.append(train_data[i, 0])
        x_train, y_train = np.array(x_train), np.array(y_train)
        test_data = scaled_data[training_data_len - 60: , :]
        x_test = []
        y_test = dataset[training_data_len:, :]
        for i in range(60, len(test_data)):
          x_test.append(test_data[i-60:i, :])
        x_test = np.array(x_test)
        x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))
        self.x_train=x_train
        self.y_train=y_train
        self.x_test=x_test
        self.y_test=y_test
        print(self.name +' : x_test shape : ',x_test.shape)
        print(self.name +' : x_train shape : ',x_train.shape)
        
    
    def plot_predict():
        onemodelnameip=self.name
        new_df = self.df.filter(['Close'])
        scaler=self.scaler
        last_60_days = new_df[-180:-120].values
        predicted_plot=last_60_days
        actual_plot=new_df[-180:].values
        try:
            last_60_days_scaled = scaler.transform(last_60_days)
        except ValueError:
            _=0
        for jk in range(120):
            X_test = []
            X_test.append(last_60_days_scaled)
            X_test = np.array(X_test)
            X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
            pred_price = self.model.predict(X_test)
            pred_price_descaled = scaler.inverse_transform(pred_price)
            predicted_plot=np.append(predicted_plot,pred_price_descaled)
            array_abcd=np.array(pred_price)
            asdasdasd=np.append(X_test,array_abcd)
            asdasdasd=np.delete(asdasdasd, 0,None)
            last_60_days_scaled = np.reshape(asdasdasd, (X_test.shape[0], X_test.shape[1], 1 ))[0]
        plot_df=pd.DataFrame(predicted_plot.T)
        plot_df2=pd.DataFrame(actual_plot)
        result = pd.concat([plot_df, plot_df2], axis=1, sort=False)
        result.plot(figsize=(11,5));
        self.plotdir=homediraddr+ '/predict_plots/'+self.name
        if(not os.path.isdir(self.plotdir)):
            os.mkdir(self.plotdir)
            print('Directory created : '+self.plotdir)
        
        filenum=0
        filecheckpath = self.plotdir + '/'+self.name+'_'+str(filenum)+'.png'
        while os.path.isfile(filecheckpath):
            filenum+=1
            filecheckpath =self.plotdir + '/'+self.name+'_'+str(filenum)+'.png'
        plt.savefig(self.plotdir + '/'+self.name+'_'+str(filenum)+'.png', dpi=100, bbox_inches='tight')

    def savemodelh5():
        modeldir=homediraddr+ '/stkwise_models/'+self.name
        if(not os.path.isdir(modeldir)):
            os.mkdir(modeldir)
            print('Directory created : '+modeldir)
        filenum=0
        filecheckpath =modeldir+'/'+self.name+'_'+str(filenum)+'.h5'
        while os.path.isfile(filecheckpath):
            filenum+=1
            filecheckpath =modeldir+'/'+self.name+'_'+str(filenum)+'.h5'
        model.save(modeldir+'/'+self.name+'_'+str(filenum)+'.h5')
        
        
        
    def fit():
        self.load()
        self.xtrain_def()
        self.plot_predict()
        history=self.model.fit(self.x_train,
                          self.y_train,
                          validation_split=0.03,
                          verbose=1,
                          batch_size=1, 
                          epochs=1,
                          callbacks=[
                                     livelossplot.PlotLossesKeras(),
                                     MyCustomCallback(self)
                                    ])
        self.plot_predict()
       
  
class MyCustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, ipobj):
        self.ipobj = ipobj
    
    counter1=0
    def on_train_batch_begin(self, batch, logs=None):
        global counter1
        counter1+=1
        if(counter1%500==0):
            self.ipobj.plot_predict()
        
        if(counter1%2500==0):
            self.ipobj.savemodelh5()
        
    def on_train_batch_end(self, batch, logs=None):
        clear_output(wait=True)      
counter1=0

In [26]:
x = One_trian_stk_class('sdfsdf', 12224546,1,789)
print(x.name, x.insttkn,x.nepochs,x.nbatch)
print(type(x))

sdfsdf 12224546 789 1
<class '__main__.One_trian_stk_class'>


In [27]:
import stkdef